# Assignment 2, 8-Puzzle Programming - A Star

### Your homework must be implemented in this Notebook file. 
### You can add as many cells as you want. However, you are not allowed to touch the code below the line "=============".
### You need to implement the three (four for grads) searching functions and the print result functions.
### For the searching functions, feel free to customize the return data types and parameter lists as long as the function name is as required.


In [1]:
###
#IMPORT PUZZLE CASES
###

puzzleCases = []
with open('Input8PuzzleCases.txt', 'r') as f:
    for rawLine in f:
        charLine = rawLine.strip('\n').split(',') #tokenize by commas, purge newlines
        intLine = [int(char) for char in charLine] #convert list members from strings to ints via list comprehension
        puzzleCases.append(intLine) #add each initial puzzle state to list as a tuple
f.close()

#global variable goalState
goalState = (0,1,2,3,4,5,6,7,8)

In [2]:
###
#GENERAL HELPER FUNCTIONS & IMPORTS
###

import time #for time metric
import heapq #heap library for frontier priority queue
import math #gotta do math

#figures out the next possible actions for a given state
#returns a dictionary mapping each action to a boolean saying whether or not the move is available at that state
def identifyActionsForState(state):
    index = findIndexOfZeroInState(state)
    #dictionary of actions available based on the position of the empty tile ("zero" tile)
    possibleActionsForZeroPosition = {
        0: {"Up": False, "Right": True, "Down": True, "Left": False},
        1: {"Up": False, "Right": True, "Down": True, "Left": True},
        2: {"Up": False, "Right": False, "Down": True, "Left": True},
        3: {"Up": True, "Right": True, "Down": True, "Left": False},
        4: {"Up": True, "Right": True, "Down": True, "Left": True},
        5: {"Up": True, "Right": False, "Down": True, "Left": True},
        6: {"Up": True, "Right": True, "Down": False, "Left": False},
        7: {"Up": True, "Right": True, "Down": False, "Left": True},
        8: {"Up": True, "Right": False, "Down": False, "Left": True}
    }
    return possibleActionsForZeroPosition[index]

#returns the index of the empty tile ("zero" tile) in a state
def findIndexOfZeroInState(state):
    index = 0
    for position in state:
        if position == 0:
            break
        index += 1
    return index

#effectively generates the children nodes for a state
def generateNextPossibleMovesForState(state):
    possibleActions = identifyActionsForState(state)
    index = findIndexOfZeroInState(state)
    nextStates = []
    #for each action available at the current state, add that action and its resulting state to a list to be returned
    if (possibleActions["Up"]):
        stateAfterMoveUp = list(state[:])
        stateAfterMoveUp[index],stateAfterMoveUp[index-3]=stateAfterMoveUp[index-3], stateAfterMoveUp[index]
        nextStates.append({"action": "Up","state": tuple(stateAfterMoveUp)})
    if (possibleActions["Right"]):
        stateAfterMoveRight = list(state[:])
        stateAfterMoveRight[index],stateAfterMoveRight[index+1]=stateAfterMoveRight[index+1], stateAfterMoveRight[index]
        nextStates.append({"action": "Right","state": tuple(stateAfterMoveRight)})
    if (possibleActions["Down"]):
        stateAfterMoveDown = list(state[:])
        stateAfterMoveDown[index],stateAfterMoveDown[index+3]=stateAfterMoveDown[index+3], stateAfterMoveDown[index]
        nextStates.append({"action": "Down","state": tuple(stateAfterMoveDown)})
    if (possibleActions["Left"]):
        stateAfterMoveLeft = list(state[:])
        stateAfterMoveLeft[index],stateAfterMoveLeft[index-1]=stateAfterMoveLeft[index-1], stateAfterMoveLeft[index]
        nextStates.append({"action": "Left","state": tuple(stateAfterMoveLeft)})
    return nextStates

#Takes a node and traces the path (in reverse) up to the initial (puzzle) state
#Used to trace the optimal solution or just a list of ancestors (useful for IDS repeated state checking)
def traceSolution(node):
    optimalSolution = []
    currentNode = node
    #trace path to initial state by pushing nodes onto a stack and tracing by parent nodes, starting at node being passed
    while currentNode != None:
        optimalSolution.append(currentNode)
        currentNode = currentNode.parent
    return optimalSolution


In [3]:
###
#HEURISTIC FUNCTIONS + ASSOCIATED HELPERS
###

#goal state defined as (0,1,2,3,4,5,6,7,8)
def calculateMisplacedTilesFromGoalState(currentState):
    misplacedTiles = 0
    for i in range(len(currentState)):
        if currentState[i] != i:
            misplacedTiles += 1
    return misplacedTiles

#goal state defined as (0,1,2,3,4,5,6,7,8)
def calculateManhattanDistanceFromGoalState(currentState):
    manhattanDistance = 0
    for i in range(len(currentState)):
        currentNumber = currentState[i]
        #if currentNumber == i, then the number is in the correct position and
        #no calculation needs to occur
        if not currentNumber == i:
            currentNumberRowColumnRepresentation = getRowColumnRepresentation(i)
            goalRowColumnRepresentation = getRowColumnRepresentation(currentNumber)
            manhattanDistance += abs(currentNumberRowColumnRepresentation[0] - goalRowColumnRepresentation[0]) #row distance
            manhattanDistance += abs(currentNumberRowColumnRepresentation[1] - goalRowColumnRepresentation[1]) #column distance
    return manhattanDistance

def getRowColumnRepresentation(number):
    position = [0,0] #row, column representation
    position[0] = number%3
    position[1] = number//3
    return tuple(position)

In [4]:
###
#DATA STRUCTURES
###

#node structure used for maintaining the explored portion of the transition model
class Node:
    def __init__(self, parent, state, action):
        self.parent = parent #parent Node
        self.state = state #state, as a tuple
        self.action = action #action, as a string

#subclass of node that tracks the depth of each node, used for DFS in IDS
class PathCostNode(Node):
    def __init__(self, parent, state, action, actualPathCost, pathCostToGoal):
        Node.__init__(self,parent,state,action)
        self.actualPathCost = actualPathCost #actual path cost
        self.pathCostToGoal = pathCostToGoal #estimated path cost to goal; determined based on heuristic function
        
    #override < operator, using total path cost (actualPathCost + pathCostToGoal) as comparison criterion
    #this is used to allow PathCostNodes to be compared directly, e.g. for heapsort
    def __lt__(self,other): #override <
        return self.getTotalPathCostToGoal() < other.getTotalPathCostToGoal()
    
    #calculate the estimated total path cost to the goal state
    def getTotalPathCostToGoal(self):
        return self.actualPathCost + self.pathCostToGoal
    
#Frontier structure used for maintaining the frontier in a more performant way than just a list used as a FIFO
#Maintains a list that is used for push(LIFO push, FIFO enqueue)/pop (LIFO)/dequeue (FIFO) operations
#Maintains a dict that is used for lookup operations
#This frontier should only be used where states are uniquely added to it; it will break if duplicate states
#are added to it
class Frontier:
    def __init__(self):
        self.list = []
        self.dict = dict()
        
    #LIFO push, FIFO enqueue
    def push(self,node):
        self.list.append(node)
        self.dict[node.state] = node
        return
    
    #FIFO dequeue
    def dequeue(self):
        poppedNode = self.list.pop(0)
        del self.dict[poppedNode.state]
        return poppedNode
    
    #LIFO pop
    def pop(self):
        poppedNode = self.list.pop()
        del self.dict[poppedNode.state]
        return poppedNode
    
    def isStateInFrontier(self,state):
        return state in self.dict
    
    def isEmpty(self):
        return len(self.list) == 0
    
#frontier subclass that overrides push and pop as heap operations, and adds heuristic functionality
#has heapq module as a dependency for heap operations
class HeapFrontier(Frontier):
    def __init__(self):
        Frontier.__init__(self)
        
    #heappush
    def push(self,node):
        heapq.heappush(self.list,node)
        self.dict[node.state] = node
        return
    
    #heappop
    def pop(self):
        poppedNode = heapq.heappop(self.list)
        del self.dict[poppedNode.state]
        return poppedNode
    
    #heappop - overriding superclass dequeue for safety
    def dequeue(self):
        return self.pop()
    
    #check if the state already exists in the frontier with a higher total path cost
    #total path cost is actual path cost + estimated distance from goal based on the pathCostFunction
    #that is passed, e.g. Misplaced Tiles or Manhattan Distance
    #takes a state and a reference to the desired heuristic function
    def isStateInFrontierWithHigherPathCost(self,state,pathCostFunction):
        if self.isStateInFrontier(state):
            if (self.dict[state].actualPathCost + pathCostFunction(state)) < self.dict[state].getTotalPathCostToGoal():
                return True
        return False
                
    #rebuilds the heap with all of the previous nodes and tries to replace
    #a node with a state that is already in the heap
    #if a node with the state of the node passed into the function exists in the heap,
    #then the node in the heap will be replaced by the new node
    #if no node with the state of the node passed into the function exists in the heap,
    #then this operation is a huge waste because it returns the same heap
    #USAGE: use after checking a frontier candidate with isStateInFrontierWithHigherPathCost()
    #if it finds that the total estimated path cost (actual + estimated) to that
    def rebuildHeapWithReplacementNode(self,newNode):
        newHeap = []
        for node in self.list:
            if node.state != newNode.state: #check whether or not this is the replacement node
                heapq.heappush(newHeap,node) #push old node
            else: #replace the node with the same state as the passed state
                self.dict.update({newNode.state: newNode})
                heapq.heappush(newHeap,newNode)
        return [heapq.heappop(newHeap) for i in range(len(newHeap))] #return a heap of 

In [5]:
#implementation of function "A Star Misplaced"

def aStarMisplaced(initialState):
    #initialize search
    frontier = HeapFrontier()
    frontier.push(PathCostNode(None,tuple(initialState),None,0,calculateMisplacedTilesFromGoalState(initialState)))
    exploredStates = set()
    currentNode = None; nodesExplored = 0
    beginningTime = time.time() #time the solution
    while True:
        currentNode = frontier.pop()
        if currentNode.state == goalState:
            break
        exploredStates.add(currentNode.state); nodesExplored += 1
        moves = generateNextPossibleMovesForState(currentNode.state)
        for move in moves:
            newNode = PathCostNode(currentNode,move["state"],move["action"],currentNode.actualPathCost+1,calculateMisplacedTilesFromGoalState(move["state"]))
            #check to see if the state is already in the frontier or exploredStates set
            if (not frontier.isStateInFrontier(move["state"])) and (move["state"] not in exploredStates):
                frontier.push(newNode) #heappush
            #check to see if the state exists in the frontier with a longer path to it
            elif frontier.isStateInFrontierWithHigherPathCost(move["state"],calculateMisplacedTilesFromGoalState):
                frontier.rebuildHeapWithReplacementNode(newNode)
    timeElapsedInSeconds = time.time() - beginningTime #track time
    optimalSolution = traceSolution(currentNode)
    return {"solution":optimalSolution,"nodesExplored":nodesExplored,"time":timeElapsedInSeconds}

In [6]:
#implementation of function "A Star Manhattan Distance"

def aStarManhattanDistance(initialState):
    frontier = HeapFrontier()
    frontier.push(PathCostNode(None,tuple(initialState),None,0,calculateManhattanDistanceFromGoalState(initialState)))
    exploredStates = set()
    currentNode = None; nodesExplored = 0
    beginningTime = time.time() #time the solution
    while True:
        currentNode = frontier.pop()
        if currentNode.state == goalState:
            break
        exploredStates.add(currentNode.state); nodesExplored += 1
        moves = generateNextPossibleMovesForState(currentNode.state)
        for move in moves:
            newNode = PathCostNode(currentNode,move["state"],move["action"],currentNode.actualPathCost+1,calculateManhattanDistanceFromGoalState(move["state"]))
            #check to see if the state is already in the frontier or exploredStates set
            if (not frontier.isStateInFrontier(move["state"])) and (move["state"] not in exploredStates):
                frontier.push(newNode) #heappush
            #check to see if the state exists in the frontier with a longer path to it
            elif frontier.isStateInFrontierWithHigherPathCost(move["state"],calculateManhattanDistanceFromGoalState):
                frontier.rebuildHeapWithReplacementNode(newNode)
    timeElapsedInSeconds = time.time() - beginningTime #track time
    optimalSolution = traceSolution(currentNode)
    return {"solution":optimalSolution,"nodesExplored":nodesExplored,"time":timeElapsedInSeconds}

In [7]:
###
#implementation of function "print_result(result)"
###

#Directly takes the output of a breadthFirstSearch(initialState) or Iterative_deepening_DFS(initialState)
#and creates a nice output showing the step by step solution for the initialState
#Example usage:
#print_result(aStarMisplaced(1,2,0,3,4,5,6,7,8)))
#print_result(aStarManhattanDistance(puzzleCases[4]))

def print_result(result):
    solution = result["solution"]
    while solution:
        step = solution.pop()
        state = step.state; action = step.action
        if action != None:
            print(action, "to")
        printStateNicely(state)
    return
    
#helper function, prints a state as a grid
def printStateNicely(state):
    print(state[0],state[1],state[2])
    print(state[3],state[4],state[5])
    print(state[6],state[7],state[8])
    return

In [8]:
###
#TESTING + METRICS CODE + OUTPUT
###

#Simple script to run each A* method on each puzzle case
#Outputs nodes explored, time taken, and steps for each example puzzle
#After running through all 100 of the example puzzles, 
#NOTE: This does NOT call print_result(result)
#NOTE: This took about an hour to run on an i7-8700k processor @3.7GHz. The summary has been included in a markdown cell below.

#setup
puzzleIndex = 0
misplacedTime = 0
manhattanTime = 0
misplacedSteps = []
manhattanSteps = []
misplacedExplored = 0
manhattanExplored = 0
for puzzle in puzzleCases:
    #Misplaced Tiles solution + metrics + puzzle output
    puzzleSolution = aStarMisplaced(puzzle)
    misplacedTime += puzzleSolution["time"]
    misplacedSteps.append(len(puzzleSolution["solution"]) - 1) #there are one fewer steps in the solution than there are elements in the solution
    misplacedExplored += puzzleSolution["nodesExplored"]
    print(puzzleIndex,"Misplaced: exploredNodes:",puzzleSolution["nodesExplored"],"time:", puzzleSolution["time"],"steps:", len(puzzleSolution["solution"]) - 1)
    #Manhattan Distance solution + metrics + puzzle output
    puzzleSolution = aStarManhattanDistance(puzzle)
    manhattanTime += puzzleSolution["time"]
    manhattanSteps.append(len(puzzleSolution["solution"]) - 1)
    manhattanExplored += puzzleSolution["nodesExplored"]
    print(puzzleIndex,"Manhattan: exploredNodes:",puzzleSolution["nodesExplored"],"time:", puzzleSolution["time"],"steps:", len(puzzleSolution["solution"]) - 1)
    #iterate index
    puzzleIndex += 1

#summary metrics
averageMisplacedTime = misplacedTime/(puzzleIndex + 1.0)
averageManhattanTime = manhattanTime/(puzzleIndex + 1.0)
totalMisplacedSteps = 0
for stepsCount in misplacedSteps:
    totalMisplacedSteps += stepsCount
totalManhattanSteps = 0
for stepsCount in manhattanSteps:
    totalManhattanSteps += stepsCount
averageMisplacedSteps = (totalMisplacedSteps + 0.0)/(puzzleIndex + 1.0)
averageManhattanSteps = (totalManhattanSteps + 0.0)/(puzzleIndex + 1.0)
averageMisplacedExplored = misplacedExplored/(puzzleIndex + 1.0)
averageManhattanExplored = manhattanExplored/(puzzleIndex + 1.0)

#print summary for A* search using Misplaced and Manhattan Distance heuristic functions over all 100 puzzle cases
print("Misplaced: ","Average Steps:",math.floor(averageMisplacedSteps),"Average Time:",averageMisplacedTime, "Average Nodes Explored:",math.floor(averageMisplacedExplored))
print("Manhattan: ","Average Steps:",math.floor(averageManhattanSteps),"Average Time:",averageManhattanTime, "Average Nodes Explored:",math.floor(averageManhattanExplored))

0 Misplaced: exploredNodes: 27269 time: 0.4013693332672119 steps: 25
0 Manhattan: exploredNodes: 4775 time: 0.13199591636657715 steps: 25
1 Misplaced: exploredNodes: 27384 time: 0.4080502986907959 steps: 25
1 Manhattan: exploredNodes: 640 time: 0.018996477127075195 steps: 25
2 Misplaced: exploredNodes: 12304 time: 0.1789994239807129 steps: 23
2 Manhattan: exploredNodes: 1408 time: 0.04102659225463867 steps: 23
3 Misplaced: exploredNodes: 26165 time: 0.3899991512298584 steps: 25
3 Manhattan: exploredNodes: 1429 time: 0.03899884223937988 steps: 25
4 Misplaced: exploredNodes: 1390 time: 0.017998933792114258 steps: 18
4 Manhattan: exploredNodes: 271 time: 0.007000446319580078 steps: 18
5 Misplaced: exploredNodes: 5345 time: 0.07102441787719727 steps: 21
5 Manhattan: exploredNodes: 653 time: 0.017972707748413086 steps: 21
6 Misplaced: exploredNodes: 65264 time: 1.0125060081481934 steps: 28
6 Manhattan: exploredNodes: 8952 time: 0.2650299072265625 steps: 28
7 Misplaced: exploredNodes: 5293 t

61 Misplaced: exploredNodes: 56884 time: 0.8700335025787354 steps: 27
61 Manhattan: exploredNodes: 6703 time: 0.19501352310180664 steps: 27
62 Misplaced: exploredNodes: 1321 time: 0.01797032356262207 steps: 18
62 Manhattan: exploredNodes: 506 time: 0.01402902603149414 steps: 18
63 Misplaced: exploredNodes: 7335 time: 0.11097073554992676 steps: 22
63 Manhattan: exploredNodes: 1445 time: 0.038997650146484375 steps: 24
64 Misplaced: exploredNodes: 15387 time: 0.21300125122070312 steps: 24
64 Manhattan: exploredNodes: 1212 time: 0.031998395919799805 steps: 24
65 Misplaced: exploredNodes: 3012 time: 0.04700016975402832 steps: 20
65 Manhattan: exploredNodes: 1250 time: 0.03199911117553711 steps: 20
66 Misplaced: exploredNodes: 64355 time: 0.9710268974304199 steps: 28
66 Manhattan: exploredNodes: 4220 time: 0.11795949935913086 steps: 28
67 Misplaced: exploredNodes: 18728 time: 0.2649998664855957 steps: 24
67 Manhattan: exploredNodes: 1478 time: 0.03999900817871094 steps: 24
68 Misplaced: expl

# Misplaced Tile and Manhattan Distance performance per puzzle + summary

## This test was performed on a MacBook Pro (Early 2015 w/ 2.7 GHz dual core + hyperthreading i5 CPU) and took under a minute to complete.

`
0 Misplaced: exploredNodes: 27269 time: 0.7987473011016846 steps: 25
0 Manhattan: exploredNodes: 4775 time: 0.20788002014160156 steps: 25
1 Misplaced: exploredNodes: 27384 time: 0.7994270324707031 steps: 25
1 Manhattan: exploredNodes: 640 time: 0.027096986770629883 steps: 25
2 Misplaced: exploredNodes: 12304 time: 0.32164573669433594 steps: 23
2 Manhattan: exploredNodes: 1408 time: 0.07225775718688965 steps: 23
3 Misplaced: exploredNodes: 26165 time: 0.6324539184570312 steps: 25
3 Manhattan: exploredNodes: 1429 time: 0.08741092681884766 steps: 25
4 Misplaced: exploredNodes: 1390 time: 0.02843618392944336 steps: 18
4 Manhattan: exploredNodes: 271 time: 0.010004997253417969 steps: 18
5 Misplaced: exploredNodes: 5345 time: 0.12183189392089844 steps: 21
5 Manhattan: exploredNodes: 653 time: 0.030541181564331055 steps: 21
6 Misplaced: exploredNodes: 65264 time: 1.6637468338012695 steps: 28
6 Manhattan: exploredNodes: 8952 time: 0.43972277641296387 steps: 28
7 Misplaced: exploredNodes: 5293 time: 0.11343097686767578 steps: 21
7 Manhattan: exploredNodes: 776 time: 0.0484309196472168 steps: 21
8 Misplaced: exploredNodes: 2611 time: 0.05540299415588379 steps: 20
8 Manhattan: exploredNodes: 785 time: 0.0314481258392334 steps: 20
9 Misplaced: exploredNodes: 2862 time: 0.07077693939208984 steps: 20
9 Manhattan: exploredNodes: 599 time: 0.02440500259399414 steps: 20
10 Misplaced: exploredNodes: 13575 time: 0.3122422695159912 steps: 24
10 Manhattan: exploredNodes: 2018 time: 0.09287905693054199 steps: 24
11 Misplaced: exploredNodes: 2217 time: 0.06268715858459473 steps: 19
11 Manhattan: exploredNodes: 289 time: 0.011306047439575195 steps: 19
12 Misplaced: exploredNodes: 5173 time: 0.12551498413085938 steps: 21
12 Manhattan: exploredNodes: 818 time: 0.03845977783203125 steps: 23
13 Misplaced: exploredNodes: 5030 time: 0.14847612380981445 steps: 21
13 Manhattan: exploredNodes: 431 time: 0.018469810485839844 steps: 21
14 Misplaced: exploredNodes: 26485 time: 0.661595344543457 steps: 25
14 Manhattan: exploredNodes: 1187 time: 0.05117988586425781 steps: 25
15 Misplaced: exploredNodes: 26473 time: 0.6347460746765137 steps: 25
15 Manhattan: exploredNodes: 5859 time: 0.27321696281433105 steps: 25
16 Misplaced: exploredNodes: 1550 time: 0.03399991989135742 steps: 18
16 Manhattan: exploredNodes: 480 time: 0.018388986587524414 steps: 18
17 Misplaced: exploredNodes: 1395 time: 0.02798008918762207 steps: 18
17 Manhattan: exploredNodes: 471 time: 0.0178377628326416 steps: 18
18 Misplaced: exploredNodes: 29853 time: 0.7233831882476807 steps: 25
18 Manhattan: exploredNodes: 2249 time: 0.0971519947052002 steps: 25
19 Misplaced: exploredNodes: 2896 time: 0.06172800064086914 steps: 20
19 Manhattan: exploredNodes: 652 time: 0.02594590187072754 steps: 20
20 Misplaced: exploredNodes: 30390 time: 0.7634978294372559 steps: 26
20 Manhattan: exploredNodes: 2020 time: 0.09063124656677246 steps: 26
21 Misplaced: exploredNodes: 11830 time: 0.28844499588012695 steps: 23
21 Manhattan: exploredNodes: 2290 time: 0.09850502014160156 steps: 23
22 Misplaced: exploredNodes: 16737 time: 0.41031789779663086 steps: 24
22 Manhattan: exploredNodes: 1505 time: 0.0635840892791748 steps: 24
23 Misplaced: exploredNodes: 12190 time: 0.30429697036743164 steps: 23
23 Manhattan: exploredNodes: 2554 time: 0.10822200775146484 steps: 23
24 Misplaced: exploredNodes: 25250 time: 0.6187758445739746 steps: 25
24 Manhattan: exploredNodes: 4546 time: 0.20054912567138672 steps: 25
25 Misplaced: exploredNodes: 4136 time: 0.1039879322052002 steps: 20
25 Manhattan: exploredNodes: 329 time: 0.013586044311523438 steps: 20
26 Misplaced: exploredNodes: 5777 time: 0.16116595268249512 steps: 21
26 Manhattan: exploredNodes: 1464 time: 0.07918596267700195 steps: 23
27 Misplaced: exploredNodes: 1626 time: 0.03928995132446289 steps: 19
27 Manhattan: exploredNodes: 264 time: 0.014252662658691406 steps: 19
28 Misplaced: exploredNodes: 64697 time: 2.6130001544952393 steps: 28
28 Manhattan: exploredNodes: 5678 time: 0.2606511116027832 steps: 28
29 Misplaced: exploredNodes: 3528 time: 0.08071303367614746 steps: 20
29 Manhattan: exploredNodes: 703 time: 0.030483007431030273 steps: 22
30 Misplaced: exploredNodes: 30278 time: 1.2632629871368408 steps: 26
30 Manhattan: exploredNodes: 2217 time: 0.09869074821472168 steps: 26
31 Misplaced: exploredNodes: 26018 time: 0.836353063583374 steps: 25
31 Manhattan: exploredNodes: 3305 time: 0.1458110809326172 steps: 25
32 Misplaced: exploredNodes: 2352 time: 0.049353837966918945 steps: 19
32 Manhattan: exploredNodes: 233 time: 0.011505126953125 steps: 19
33 Misplaced: exploredNodes: 11373 time: 0.2824709415435791 steps: 23
33 Manhattan: exploredNodes: 747 time: 0.036135196685791016 steps: 23
34 Misplaced: exploredNodes: 6707 time: 0.15763616561889648 steps: 22
34 Manhattan: exploredNodes: 566 time: 0.024637937545776367 steps: 22
35 Misplaced: exploredNodes: 14767 time: 0.3826920986175537 steps: 24
35 Manhattan: exploredNodes: 5255 time: 0.35026979446411133 steps: 26
36 Misplaced: exploredNodes: 4970 time: 0.10916590690612793 steps: 21
36 Manhattan: exploredNodes: 694 time: 0.027767181396484375 steps: 21
37 Misplaced: exploredNodes: 31207 time: 0.8534688949584961 steps: 26
37 Manhattan: exploredNodes: 6934 time: 0.3421759605407715 steps: 26
38 Misplaced: exploredNodes: 5121 time: 0.12343883514404297 steps: 21
38 Manhattan: exploredNodes: 419 time: 0.022665023803710938 steps: 21
39 Misplaced: exploredNodes: 5836 time: 0.20558905601501465 steps: 22
39 Manhattan: exploredNodes: 1014 time: 0.04566001892089844 steps: 22
40 Misplaced: exploredNodes: 55338 time: 1.87492036819458 steps: 27
40 Manhattan: exploredNodes: 5793 time: 0.51059889793396 steps: 27
41 Misplaced: exploredNodes: 14329 time: 0.6074461936950684 steps: 24
41 Manhattan: exploredNodes: 1942 time: 0.10589814186096191 steps: 24
42 Misplaced: exploredNodes: 196 time: 0.003467082977294922 steps: 14
42 Manhattan: exploredNodes: 97 time: 0.0035419464111328125 steps: 14
43 Misplaced: exploredNodes: 50037 time: 1.878335952758789 steps: 27
43 Manhattan: exploredNodes: 3940 time: 0.18625497817993164 steps: 27
44 Misplaced: exploredNodes: 11923 time: 0.2919578552246094 steps: 23
44 Manhattan: exploredNodes: 1501 time: 0.06167483329772949 steps: 23
45 Misplaced: exploredNodes: 2914 time: 0.05999422073364258 steps: 20
45 Manhattan: exploredNodes: 365 time: 0.01454615592956543 steps: 20
46 Misplaced: exploredNodes: 13650 time: 0.33454084396362305 steps: 23
46 Manhattan: exploredNodes: 2630 time: 0.1103670597076416 steps: 23
47 Misplaced: exploredNodes: 2802 time: 0.05814194679260254 steps: 20
47 Manhattan: exploredNodes: 327 time: 0.013190031051635742 steps: 20
48 Misplaced: exploredNodes: 2507 time: 0.051769256591796875 steps: 19
48 Manhattan: exploredNodes: 255 time: 0.009971380233764648 steps: 19
49 Misplaced: exploredNodes: 6154 time: 0.17775988578796387 steps: 22
49 Manhattan: exploredNodes: 1083 time: 0.043565988540649414 steps: 22
50 Misplaced: exploredNodes: 13488 time: 0.3236711025238037 steps: 24
50 Manhattan: exploredNodes: 1026 time: 0.043514251708984375 steps: 24
51 Misplaced: exploredNodes: 20425 time: 0.5330708026885986 steps: 24
51 Manhattan: exploredNodes: 1005 time: 0.04445481300354004 steps: 24
52 Misplaced: exploredNodes: 6338 time: 0.2256300449371338 steps: 22
52 Manhattan: exploredNodes: 1000 time: 0.04947519302368164 steps: 22
53 Misplaced: exploredNodes: 41 time: 0.0013859272003173828 steps: 11
53 Manhattan: exploredNodes: 23 time: 0.0014679431915283203 steps: 11
54 Misplaced: exploredNodes: 11877 time: 0.844836950302124 steps: 23
54 Manhattan: exploredNodes: 942 time: 0.04263782501220703 steps: 23
55 Misplaced: exploredNodes: 1233 time: 0.02533698081970215 steps: 18
55 Manhattan: exploredNodes: 628 time: 0.025573253631591797 steps: 20
56 Misplaced: exploredNodes: 5177 time: 0.12033915519714355 steps: 21
56 Manhattan: exploredNodes: 975 time: 0.0480799674987793 steps: 21
57 Misplaced: exploredNodes: 5232 time: 0.13193082809448242 steps: 21
57 Manhattan: exploredNodes: 714 time: 0.0290219783782959 steps: 23
58 Misplaced: exploredNodes: 11705 time: 0.3112330436706543 steps: 23
58 Manhattan: exploredNodes: 1704 time: 0.17064905166625977 steps: 23
59 Misplaced: exploredNodes: 2462 time: 0.21722698211669922 steps: 19
59 Manhattan: exploredNodes: 618 time: 0.05402708053588867 steps: 19
60 Misplaced: exploredNodes: 7051 time: 0.18906593322753906 steps: 22
60 Manhattan: exploredNodes: 473 time: 0.022161006927490234 steps: 22
61 Misplaced: exploredNodes: 56884 time: 1.9252870082855225 steps: 27
61 Manhattan: exploredNodes: 6703 time: 0.5158448219299316 steps: 27
62 Misplaced: exploredNodes: 1321 time: 0.06485414505004883 steps: 18
62 Manhattan: exploredNodes: 506 time: 0.02431797981262207 steps: 18
63 Misplaced: exploredNodes: 7335 time: 0.18794989585876465 steps: 22
63 Manhattan: exploredNodes: 1445 time: 0.06997370719909668 steps: 24
64 Misplaced: exploredNodes: 15387 time: 0.3569636344909668 steps: 24
64 Manhattan: exploredNodes: 1212 time: 0.0500788688659668 steps: 24
65 Misplaced: exploredNodes: 3012 time: 0.08370709419250488 steps: 20
65 Manhattan: exploredNodes: 1250 time: 0.05203986167907715 steps: 20
66 Misplaced: exploredNodes: 64355 time: 1.7121860980987549 steps: 28
66 Manhattan: exploredNodes: 4220 time: 0.18774008750915527 steps: 28
67 Misplaced: exploredNodes: 18728 time: 0.4502389430999756 steps: 24
67 Manhattan: exploredNodes: 1478 time: 0.06385207176208496 steps: 24
68 Misplaced: exploredNodes: 55416 time: 1.3671739101409912 steps: 28
68 Manhattan: exploredNodes: 9380 time: 0.43826913833618164 steps: 28
69 Misplaced: exploredNodes: 55249 time: 1.3835031986236572 steps: 27
69 Manhattan: exploredNodes: 13122 time: 0.6068217754364014 steps: 29
70 Misplaced: exploredNodes: 6590 time: 0.15845227241516113 steps: 22
70 Manhattan: exploredNodes: 1898 time: 0.08332180976867676 steps: 24
71 Misplaced: exploredNodes: 3093 time: 0.06444096565246582 steps: 20
71 Manhattan: exploredNodes: 902 time: 0.03932523727416992 steps: 20
72 Misplaced: exploredNodes: 15379 time: 0.3773031234741211 steps: 24
72 Manhattan: exploredNodes: 2707 time: 0.11654400825500488 steps: 26
73 Misplaced: exploredNodes: 15837 time: 0.3801310062408447 steps: 24
73 Manhattan: exploredNodes: 2232 time: 0.09463191032409668 steps: 24
74 Misplaced: exploredNodes: 25703 time: 0.6130821704864502 steps: 25
74 Manhattan: exploredNodes: 4669 time: 0.20484709739685059 steps: 25
75 Misplaced: exploredNodes: 14661 time: 0.35828399658203125 steps: 24
75 Manhattan: exploredNodes: 2742 time: 0.11216592788696289 steps: 24
76 Misplaced: exploredNodes: 13966 time: 0.3139169216156006 steps: 24
76 Manhattan: exploredNodes: 2839 time: 0.1192319393157959 steps: 24
77 Misplaced: exploredNodes: 5174 time: 0.13234996795654297 steps: 21
77 Manhattan: exploredNodes: 542 time: 0.021967172622680664 steps: 21
78 Misplaced: exploredNodes: 2605 time: 0.05497407913208008 steps: 19
78 Manhattan: exploredNodes: 627 time: 0.024633169174194336 steps: 19
79 Misplaced: exploredNodes: 24857 time: 0.6018087863922119 steps: 25
79 Manhattan: exploredNodes: 2007 time: 0.09139609336853027 steps: 25
80 Misplaced: exploredNodes: 5490 time: 0.11534595489501953 steps: 21
80 Manhattan: exploredNodes: 552 time: 0.022258996963500977 steps: 21
81 Misplaced: exploredNodes: 4865 time: 0.11105990409851074 steps: 21
81 Manhattan: exploredNodes: 538 time: 0.022015094757080078 steps: 21
82 Misplaced: exploredNodes: 9219 time: 0.22789883613586426 steps: 22
82 Manhattan: exploredNodes: 793 time: 0.03221416473388672 steps: 22
83 Misplaced: exploredNodes: 12133 time: 0.27094507217407227 steps: 23
83 Manhattan: exploredNodes: 5370 time: 0.2619740962982178 steps: 27
84 Misplaced: exploredNodes: 6168 time: 0.1335768699645996 steps: 22
84 Manhattan: exploredNodes: 1501 time: 0.06264305114746094 steps: 22
85 Misplaced: exploredNodes: 14347 time: 0.3437538146972656 steps: 23
85 Manhattan: exploredNodes: 3399 time: 0.14814019203186035 steps: 23
86 Misplaced: exploredNodes: 2312 time: 0.04803776741027832 steps: 19
86 Manhattan: exploredNodes: 322 time: 0.01270604133605957 steps: 19
87 Misplaced: exploredNodes: 21575 time: 0.5530600547790527 steps: 24
87 Manhattan: exploredNodes: 2183 time: 0.10281133651733398 steps: 24
88 Misplaced: exploredNodes: 12044 time: 0.27887797355651855 steps: 23
88 Manhattan: exploredNodes: 2320 time: 0.11111617088317871 steps: 23
89 Misplaced: exploredNodes: 26855 time: 0.6421718597412109 steps: 25
89 Manhattan: exploredNodes: 7058 time: 0.3118572235107422 steps: 27
90 Misplaced: exploredNodes: 25974 time: 0.6515250205993652 steps: 25
90 Manhattan: exploredNodes: 5972 time: 0.26966118812561035 steps: 25
91 Misplaced: exploredNodes: 11540 time: 0.27323412895202637 steps: 23
91 Manhattan: exploredNodes: 1256 time: 0.05356287956237793 steps: 23
92 Misplaced: exploredNodes: 11523 time: 0.2784390449523926 steps: 23
92 Manhattan: exploredNodes: 717 time: 0.03460383415222168 steps: 25
93 Misplaced: exploredNodes: 1216 time: 0.02509784698486328 steps: 18
93 Manhattan: exploredNodes: 637 time: 0.024396896362304688 steps: 20
94 Misplaced: exploredNodes: 1376 time: 0.029169797897338867 steps: 18
94 Manhattan: exploredNodes: 644 time: 0.025435924530029297 steps: 20
95 Misplaced: exploredNodes: 6247 time: 0.15450501441955566 steps: 21
95 Manhattan: exploredNodes: 1509 time: 0.06177210807800293 steps: 21
96 Misplaced: exploredNodes: 2573 time: 0.05394101142883301 steps: 19
96 Manhattan: exploredNodes: 352 time: 0.015244007110595703 steps: 19
97 Misplaced: exploredNodes: 5730 time: 0.1332399845123291 steps: 21
97 Manhattan: exploredNodes: 574 time: 0.024873971939086914 steps: 21
98 Misplaced: exploredNodes: 13740 time: 0.33210277557373047 steps: 24
98 Manhattan: exploredNodes: 1561 time: 0.0643467903137207 steps: 24
99 Misplaced: exploredNodes: 56906 time: 1.553412675857544 steps: 27
99 Manhattan: exploredNodes: 7105 time: 0.4309418201446533 steps: 27
Misplaced:  Average Steps: 22 Average Time: 0.416863755424424 Average Nodes Explored: 14926
Manhattan:  Average Steps: 22 Average Time: 0.10187379676516693 Average Nodes Explored: 2072
`

You can insert as many cells as you want above
You are not Allowed to modify the code below this line.
## ===============================

In [ ]:
#you need to implement print_result function to print out the result according to the required format
print_result(result)


# The output format should be as follows. You only need to give one sample solution as an example.
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
# Solution of the first Scenario:
#### X X X
#### X X X
#### X X X
#### to
#### X X X
#### X X X
#### X X X
#### to
#### X X X
#### X X X
#### X X X
#### to
#### X X X
#### X X X
#### X X X
#### to
#### .
#### .
#### .
#### 0 1 2
#### 3 4 5
#### 6 7 8

                    Average_Steps    Average_Time      
 Miss Placed
 
 Manhattan Distance

